# CNN Classifier

This notebook is for training a CNN

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import config as conf
import random

# Set the random seeds for reproducibility
seed_value = 42
random.seed(seed_value)
tf.random.set_seed(seed_value)

## Load Images

Images are loaded from directories specified in ```config.py```

Image size is also specified at this point, so make sure that ```IMAGE_SIZE``` is to the specified size of the images within your train and validate directories.

In [2]:
def load_images():
    """ Loads the images which will be used for training.

    Returns:
        train, validate: Preprocessed train, validation data which will be used for training
    """

    # Normalise pixel values
    train_datagen = ImageDataGenerator(rescale=1./255) 
    validation_datagen = ImageDataGenerator(rescale=1./255)

    # Load images from the directories
    train = train_datagen.flow_from_directory(
        conf.TRAIN_DIR,
        target_size=(conf.IMAGE_SIZE, conf.IMAGE_SIZE),
        color_mode='grayscale',
        batch_size=conf.BATCH_SIZE,
        class_mode='categorical'
    )

    validate = validation_datagen.flow_from_directory(
        conf.VALIDATION_DIR,
        target_size=(conf.IMAGE_SIZE, conf.IMAGE_SIZE),
        color_mode='grayscale', 
        batch_size=conf.BATCH_SIZE,
        class_mode='categorical'
    )

    return train, validate

train_data, validation_data = load_images()

Found 41431 images belonging to 2 classes.
Found 10359 images belonging to 2 classes.


## Create Model

Creates a model with architecture specified within the function. This can be changed depending on the model architecture you want to train on.

Parameters ```IMAGE_SIZE``` and ```IMAGE_CHANNELS``` are specified within ```config.py``` and should match the images which the model will be trained on.

Parameters ```LEARNING_RATE``` and ```DROPOUT``` can be modified in ```config.py``` as well, with a dropout layer only being added if ```DROPOUT_BOOL``` is true.

In [3]:
def create_model():
    """Creates the CNN model based on the provided architecture.
    
    Returns:
        model : The created CNN model
    """
    
    model = models.Sequential()
    
    # Input layer
    model.add(layers.InputLayer(shape=(conf.IMAGE_SIZE, conf.IMAGE_SIZE, conf.IMAGE_CHANNELS)))
    
    # Model architecture
    layer_list = [
        layers.Conv2D(32, (3, 3), activation='relu'),  # Conv Layer 1
        layers.MaxPooling2D((2, 2)),  # Max Pooling 1
        layers.Conv2D(64, (3, 3), activation='relu'),  # Conv Layer 2
        layers.MaxPooling2D((2, 2)),  # Max Pooling 2
        layers.Conv2D(64, (3, 3), activation='relu'),  # Conv Layer 3
        layers.Flatten(),
        layers.Dense(64, activation='relu')
    ]
    
    # Add layers from the list
    for layer in layer_list:
        model.add(layer)
    
    # Add dropout layer if DROPOUT_BOOL is True
    if conf.DROPOUT_BOOL:
        model.add(layers.Dropout(conf.DROPOUT))
    
    # Output layer
    model.add(layers.Dense(2, activation='softmax'))
    
    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=conf.LEARNING_RATE)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

model = create_model()

## Train CNN

Model is trained and saved to directory specified in ```config.py```

In [4]:
# Trains the model
history = model.fit(
    train_data,
    epochs=conf.EPOCHS,
    validation_data=validation_data
)

# Saves model, to file specified in config
model.save(conf.MODEL_SAVE_PATH)

Epoch 1/5


/Users/oscarklemenz/Documents/All Uni Modules/Year 3/Dissertation/Code/Star-Cluster-Classifier/.venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1295/1295 ━━━━━━━━━━━━━━━━━━━━ 89s 68ms/step - accuracy: 0.8823 - loss: 0.2652 - val_accuracy: 0.9557 - val_loss: 0.0959
Epoch 2/5
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 420s 325ms/step - accuracy: 0.9533 - loss: 0.1028 - val_accuracy: 0.9616 - val_loss: 0.0816
Epoch 3/5
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 88s 68ms/step - accuracy: 0.9645 - loss: 0.0846 - val_accuracy: 0.9735 - val_loss: 0.0704
Epoch 4/5
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 89s 69ms/step - accuracy: 0.9737 - loss: 0.0655 - val_accuracy: 0.9734 - val_loss: 0.0653
Epoch 5/5
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 91s 70ms/step - accuracy: 0.9760 - loss: 0.0610 - val_accuracy: 0.9748 - val_loss: 0.0641
